In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
dataset=pd.read_csv('CKD.csv')

In [2]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [3]:
dataset.columns


Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [4]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset['classification_yes']

In [5]:
dataset["classification_yes"].value_counts()


classification_yes
1    249
0    150
Name: count, dtype: int64

In [6]:
#split into training set and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [7]:
#preprocessig the dataset
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
print(x_train)


[[ 0.48681432  1.79924415 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [ 0.30305399  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-0.61574769 -0.50842064 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 ...
 [-0.43198735  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-1.35078903  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-0.06446668 -1.27764223 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]]


In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(x_train, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Lenovo\anaconda3\Lib\site-pack

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [9]:
# print best parameter after tuning 
print(grid.best_params_) 


{'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'random'}


In [10]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00800657, 0.00240469, 0.40148988, 0.23556271, 0.00363111,
       0.00431151, 0.        , 0.00174484, 0.00531135, 0.00685759,
       0.00580444, 0.00454798]), 'std_fit_time': array([0.        , 0.00320399, 0.00842318, 0.18824237, 0.00362262,
       0.00428991, 0.        , 0.00292272, 0.00457502, 0.00781511,
       0.00710896, 0.00257629]), 'mean_score_time': array([0.        , 0.        , 0.03020406, 0.02053237, 0.01009684,
       0.00324593, 0.        , 0.        , 0.00431809, 0.00373168,
       0.00954943, 0.00724621]), 'std_score_time': array([0.        , 0.        , 0.00160456, 0.01292566, 0.00283722,
       0.0038932 , 0.        , 0.        , 0.00396173, 0.00673185,
       0.00550445, 0.00190427]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False, False, Fa

In [11]:
grid_predictions = grid.predict(x_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm




array([[51,  0],
       [ 0, 82]], dtype=int64)

In [12]:


# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        51
           1       1.00      1.00      1.00        82

    accuracy                           1.00       133
   macro avg       1.00      1.00      1.00       133
weighted avg       1.00      1.00      1.00       133



In [13]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

1.0

In [27]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008007,0.000000,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.002405,0.003204,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.401490,0.008423,0.030204,0.001605,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.944707,0.943699,0.981217,0.943651,0.981217,0.958898,0.018227,4
3,0.235563,0.188242,0.020532,0.012926,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.981569,0.981233,0.944023,0.981217,0.981217,0.973852,0.014915,2
4,0.003631,0.003623,0.010097,0.002837,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.963284,0.924528,0.962573,0.961826,0.961826,0.954807,0.015149,5
5,0.004312,0.004290,0.003246,0.003893,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.981569,0.961755,0.981217,0.962264,0.981217,0.973604,0.009469,3
6,0.000000,0.000000,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001745,0.002923,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.005311,0.004575,0.004318,0.003962,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.926978,0.925272,0.962264,0.943651,0.962264,0.944086,0.016172,7
9,0.006858,0.007815,0.003732,0.006732,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.981569,0.981014,0.981031,0.981217,0.981217,0.981210,0.000200,1
